# Google Gemini: Precise File Editing

This notebook demonstrates how to use Google's Gemini models for precise code and document editing tasks. We'll leverage Gemini's long context and instruction following ability to make targeted modifications using a diff-like format inspired by [Aider](https://github.com/paul-gauthier/aider).

We'll use a `diff-fenced` format that clearly shows the changes to be made. The “diff-fenced” edit format asks the LLM to specify file edits as a series of search/replace blocks with the file path is placed inside the fence.

```
mathweb/flask/app.py
<<<<<<< SEARCH
from flask import Flask
=======
import math
from flask import Flask
>>>>>>> REPLACE
```

If you don't have an API key, you can get one for free from [Google AI Studio](https://aistudio.google.com/app/apikey).


In [ ]:
!pip install google-genai

In [1]:
import re 
from google import genai

# create client
client = genai.Client()
model_id = "gemini-2.5-pro" # "gemini-2.5-flash"

SYSTEM_PROMPT = """
You are an expert editing assistant. You are might receive or multiple files with user instruction to make changes to them using a diff-fenced format. 

Files are presented with their relative path followed by code fence markers and the complete file content:

## How to make Edits (diff-fenced format):
When making changes, you MUST use the SEARCH/REPLACE block format as follows:

1. Basic Format Structure
```diff
filename.py
<<<<<<< SEARCH  
// original text that should be found and replaced  
=======  
// new text that will replace the original content  
>>>>>>> REPLACE  
```
  
2. Format Rules: 
- The first line must be a code fence opening marker (```diff)  
- The second line must contain ONLY the file path, exactly as shown to you  
- The SEARCH block must contain the exact content to be replaced  
- The REPLACE block contains the new content  
- End with a code fence closing marker (```)  
- Include enough context in the SEARCH block to uniquely identify the section to change  
- Keep SEARCH/REPLACE blocks concise - break large changes into multiple smaller blocks  
- For multiple changes to the same file, use multiple SEARCH/REPLACE blocks  
  
3. **Creating New Files**: Use an empty SEARCH section:  

```diff
new_file.py
<<<<<<< SEARCH  
=======  
# New file content goes here  
def new_function():  
    return "Hello World"  
>>>>>>> REPLACE
``` 
4. **Moving Content**: Use two SEARCH/REPLACE blocks:  1. One to delete content from its original location (empty REPLACE section). 2. One to add it to the new location (empty SEARCH section)  

5. **Multiple Edits**: Present each edit as a separate SEARCH/REPLACE block  

```diff
math_utils.py
<<<<<<< SEARCH  
def factorial(n):  
    if n == 0:  
        return 1  
    else:  
        return n * factorial(n-1)  
=======  
import math  
  
def factorial(n):  
    return math.factorial(n)  
>>>>>>> REPLACE

```diff
app.py
<<<<<<< SEARCH  
from utils import helper  
=======  
from utils import helper  
import math_utils  
>>>>>>> REPLACE  
  
## Important Guidelines  
  
1. Always include the EXACT file path as shown in the context  
2. Make sure the SEARCH block EXACTLY matches the existing content  
3. Break large changes into multiple smaller, focused SEARCH/REPLACE blocks  
4. Only edit files that have been added to the context  
5. Explain your changes before presenting the SEARCH/REPLACE blocks  
6. If you need to edit files not in the context, ask the user to add them first  
  
Following these instructions will ensure your edits can be properly applied to the document.  
"""

def apply_edits(original_text, diff_text):
    """Apply edits from a diff-fenced format to the original text"""
    # Extract edit blocks from the diff-fenced format
    pattern = r'```diff\n(.*?)\n<<<<<<< SEARCH\n(.*?)=======\n(.*?)>>>>>>> REPLACE\n```'
    edit_blocks = re.findall(pattern, diff_text, re.DOTALL)
    
    if not edit_blocks:
        # Try without the code fences for direct input
        pattern = r'(.*?)\n<<<<<<< SEARCH\n(.*?)=======\n(.*?)>>>>>>> REPLACE'
        edit_blocks = re.findall(pattern, diff_text, re.DOTALL)
        
    if not edit_blocks:
        return original_text
    
    # Apply each edit block
    result = original_text
    for block in edit_blocks:
        if len(block) == 3:
            filename, search_text, replace_text = block
            # Strip any leading/trailing whitespace
            search_text = search_text.strip()
            replace_text = replace_text.strip()
            
            # Replace the search text with the replace text
            if search_text in result:
                result = result.replace(search_text, replace_text)
            else:
                # If search text not found and it's empty, append the replace text
                if not search_text.strip():
                    result += '\n' + replace_text
    
    return result


In [2]:
# Instruction and file to edit
instruction = "Update the budget for phase 2 to 10% more"
file = "../assets/Project_Chimera_Status_Report_Oct2023.md"

# Generate the edits based on the instruction
response = client.models.generate_content(
    model=model_id,
    contents=f"```{file}\n{open(file).read()}\n```\n\nInstruction: {instruction}",
    config={
        "system_instruction":SYSTEM_PROMPT,    
    },
    
)

In [3]:
print(response.text)

Of course. I will update the Project Chimera status report to reflect the change in the Phase 2 budget variance.

I am updating the Phase 2 budget variance mentioned in the "Challenges and Risks" section from 15% to 10% higher than the initial budget, as requested.

```diff
../assets/Project_Chimera_Status_Report_Oct2023.md
<<<<<<< SEARCH
*   **Phase 2 Budget Variance (Medium Risk):** Initial detailed cost estimates for Phase 2 cloud infrastructure and necessary third-party API licenses are tracking approximately 15% higher than the initial budget allocated.
*   **Phase 2 Budget Variance (Medium Risk):** Initial detailed cost estimates for Phase 2 cloud infrastructure and necessary third-party API licenses are tracking approximately 10% higher than the initial budget allocated.
>>>>>>> REPLACE
```


In [4]:
# Apply the edits to the file
with open(file, "w") as f:
    applied_edits = apply_edits(open(file).read(), response.text)
    f.write(applied_edits)

In [5]:
print(open(file).read())